# <center> **$\text{Проектирование валидации. Baseline}$** </center>
----

## $\texttt{Оглавление}$
1. [$\Large{\texttt{Проектирование валидации}}$](#introduction) 
    * 1. [$\texttt{Формирование обучающей выборки}$](#problem)
    * 2. [$\texttt{Проектирование схемы валидации с учетом специфики задачи }$](#problem-formulation)
    * 3. [$\texttt{Обоснование способа выбора валидации}$](#data-analysis)
2. [$\Large{\texttt{Baseline}}$](#solution)
    * 1. [$\texttt{Выбор метрики оценки качества и обоснование выбора}$](#existing-solution)
    * 2. [$\texttt{Разработка baseline}$](#property-solution)
    * 3. [$\texttt{Реализация выбранных решиений }$](#choice-solution)
    * 4. [$\texttt{Тесты работы baseline}$](#test-solution)
    * 5. [$\texttt{Выбор итогового решения для дальнейшей оптимизации и обоснования выбора}$](#result-solution)


# <a id="introduction"> </a>$\text{Проектирование валидации}$

## <a id="problem"> </a>$\text{ Формирование обучающей выборки }$

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

На этапе отбора кандидатов будут использоваться подходы:
 - Popular Baseline 
 - Historical Baseline 
 - User-to-User 
 - LightFM 
 - Sequential - GRU4Rec

На втором этапе по результатам работы моделей первого этапа будет использоваться `CatBoostClassifier`


</div>

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

Совместная работа двух этапов модели является выходными данными рекомендательной системы, для проверки точности которых выборка будет делиться на глобальные трейн и тест. Чтобы наилучшим образом приблизить действительные условия использования модели, имеющиеся данные разбиваются по времени, а холодные пользователи остаются в тестовом ноборе. 
</div>

In [4]:
!pip install polars loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


import numpy as np
import polars as pl
import pandas as pd
import dill
import scipy
from __future__ import annotations
from datetime import datetime as dt


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from scipy.sparse import csr_matrix, coo_matrix
from polars.testing import assert_frame_equal
from typing import Tuple, Iterable, Dict
from scipy import sparse as sp
from loguru import logger
from polars.type_aliases import SchemaDefinition
from numpy.random import shuffle

from abc import ABC, abstractmethod
from scipy.sparse import csr_matrix

import random
import os


/tmp/ipykernel_35/2995659135.py:21: DeprecationWarning: The `polars.type_aliases` module is deprecated. The type aliases have moved to the `polars._typing` module to explicitly mark them as private. Please define your own type aliases, or temporarily import from the `polars._typing` module. A public `polars.typing` module will be added in the future.
  from polars.type_aliases import SchemaDefinition


In [7]:
!pip install recbole torch ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 33.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 25.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import logging
from logging import getLogger

from typing import List, Tuple

from collections import defaultdict

lst = !pip list
avail_libs = set(x.split()[0] for x in lst)
if 'recbole' not in avail_libs:
    !pip install recbole
if 'torch' not in avail_libs:
    !pip install torch
if 'ray' not in avail_libs:
    !pip install ray

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_logger
from recbole.utils.case_study import full_sort_topk
import torch

DEVICE = torch.device("cuda") if torch.cuda.is_available()\
else torch.device("cpu")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.8 MB/s eta 0:00:0000:01
  Attempting uninstall: colorlog
    Found existing installation: colorlog 6.8.2
    Uninstalling colorlog-6.8.2:
      Successfully uninstalled colorlog-6.8.2
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.6
    Uninstalling colorama-0.4.6:
      Successfully uninstalled colorama-0.4.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bayesian-optimization 1.5.1 requires colorama<0.5.0,>=0.4.6, but you have colorama 0.4.4 which is incompatible.


In [3]:
torch.cuda.is_available()

True

In [4]:
seed = 42
np.random.seed(seed)
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
    

In [5]:
data_path1 = "/kaggle/input/wb-pract/data.csv"

In [6]:
data = pl.read_csv(data_path1)

In [7]:
data = data.with_columns(order_ts = data["order_ts"].str.to_datetime())
data.head()

user_id,item_id,order_ts
i64,i64,datetime[μs]
550614,264,2023-01-01 00:28:09
571051,580,2023-01-01 00:41:47
571051,180,2023-01-01 00:41:47
47164,5135,2023-01-01 00:53:35
219072,2668,2023-01-01 01:02:29


In [41]:
data.shape

(21265784, 3)

In [38]:
del data

In [39]:
import gc
gc.collect()

0

In [7]:
data["order_ts"].min(), data["order_ts"].max()

(datetime.datetime(2023, 1, 1, 0, 0, 5),
 datetime.datetime(2023, 3, 31, 23, 59, 59, 947831))

In [9]:
global_split = dt(year=2023,month=3,day=15)
train_global = data.filter(pl.col("order_ts") < global_split ) # first 75 days
test_global = data.filter(pl.col("order_ts") >= global_split ) # last 15 days

In [40]:
train_global.shape, test_global.shape

((272593, 3), (4212490, 3))

In [ ]:
del train_global
del global_split

In [35]:
test_global.write_csv('test_global.csv')

## <a id="problem-formulation"> </a>$\text{Проектирование схемы валидации с учетом специфики задачи}$

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

Тренеровачная выборка будет разделена по времени, что аналогично отбору некоторого количества последних покупок пользователя. Более ранняя часть используеся для обучения моделей первого этапа (отбора кандидатов). Более поздняя часть - для пердсказания первого этапа, на которых в последствии обучается `CatBoostClassifier`. Для валидации классификатора эти предсказания делятся на еще одни трейн и тест. 

</div>

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

Схема будет немного отличаться для бучения последовательной модели. Реализация из модуля `RecBole` тебует специального формата данных `Atomic`, а сама реализация валидации закладывается в модель через конфиг.  
Через этот конфиг можно настроить похожее ребиение данных с использоавнием  leave one out валидации. Выбор основается на том чтобы не усложнять процесс обучения. 

```
 "eval_args": {
        "group_by": "user", # группируем по пользователям
        "order": "TO", # сортируем по timestamp-у
        "split": {"LS": "valid_only"}, # leave-one-out validation
        "mode": "full", # используем все данные для inference
    }
```
Последовательные рекомендательные модели основываются на рекурентных сетях, что  требует больших вычислительных ресурсов, тем более на всем множестве имеющихся в датасете уникальных пользователях и товарах. Поэтому возможно из данных передаваемых в модель придется исключить самах неактивных покупателей. Это решение основывается на том, что на количество параметров модели вилияют количество эбедингов всех пользователй, а не их размер. 
</div>

In [10]:
train_rerank_split = dt(year=2023,month=3,day=1)
base_data = data.filter(pl.col("order_ts") < train_rerank_split ) # first 60 days
candidate_predict_set = data.filter(pl.col("order_ts") >= train_rerank_split) # [60, 75] days

base_model_split = dt(year=2023,month=2,day=20)
train_base = data.filter(pl.col("order_ts") < base_model_split) # first 50 days
val_base = data.filter(pl.col("order_ts") >= base_model_split)  # [50, 60] days
train_val_base = base_data

In [12]:
train_base.write_csv('train_base.csv')
val_base.write_csv('val_base.csv')
train_val_base.write_csv('train_val_base.csv')

In [13]:
train_base.shape, val_base.shape, candidate_predict_set.shape

((11102479, 3), (10163305, 3), (7926469, 3))

In [41]:
del train_base
del val_base
del candidate_predict_set

In [11]:
# First, get a unique list of users
users = candidate_predict_set['user_id'].unique()

# Split users into train and test sets
users_train, users_test = train_test_split(users, test_size=0.2, random_state=42)

# Filter the rows of the DataFrame based on the split user sets
df = candidate_predict_set
train_rerank = df.filter(pl.col("user_id").is_in(users_train))
test_rerank = df.filter(pl.col("user_id").is_in(users_test))

In [40]:
train_rerank.write_csv('train_rerank.csv')
test_rerank.write_csv('test_rerank.csv')

In [41]:
candidate_predict_set.write_csv('candidate_predict_set.csv')

In [15]:
train_rerank.shape, test_rerank.shape

((6343598, 3), (1582871, 3))

In [12]:
debug_set = train_global = data.filter(pl.col("order_ts") <  dt(year=2023,month=1,day=3) ) # first 2 days

In [11]:
del d

In [42]:
import gc
gc.collect()

0

## <a id="data-analysis"> </a>$\text{Обоснование способа выбора валидации}$

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

- Очень большУю часть информации в датасете несет фича времени, поэтому для ее правильного использования важно сохронять хронологичесчий порядок действий пользователей. Это учтено при разбиении на трейн и тест. 
- При оценке последовательных моделей рекомендаций важно сохранить временной порядок взаимодействий. Метод leave-one-out валидации позволяет учитывать последовательность действий пользователя и оценивать точность модели на основе того, насколько хорошо она может предсказать следующие действия на основе уже сделанных пользователем.
- Совместно с разделенеим на тренировочный и тестовый набры построен валидационный набор для настройки гиперпараметров и выбора модели.
- Для дополнительной настройкии гипер параметров возможно Expanding Window Validation. 

</div>

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

*Стоит отметить, что пользователи с одной покупкой были оставленны и в обучающей и тренировачной выборке. Такие пользователи будут ухудшать показания моделей на метриках, но с другой стороны приближать условия тестирования и обучения к настоищим, иметируя проблему холодного старта*

# <a id="solution"> </a>$\text{Baseline}$ 

## <a id="existing-solution"> </a>$\text{Выбор метрики оценки качества и обоснование выбора}$

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

Итогом работы рекомендательной системы является заполнение страници релевантными товарами. Поскольку на одном фрейме без прокрутки помещается 6 товаров, то можно оценивать метрики для топ 6 рекомендованных. Также возможно учесть и следующие 6 товаров, считая метрики для первых 12-ти. В этой работе я буду придерживаться диапазона 6 - 12 товаров, из расчета скорости работы. 

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

Метрики в рекомендательных системах можно разделить на следующие группы:
- Регрессионные
- Классификационные
- Ранжирующие

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

Также существуют продчвинутые метрики:
- Coverage - покрытие
- Diversity - разнообразие
- Novelty - новизна
- Serendipity - прозорливость

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">

Такие метрики как **MAP@k** и **NDCG@k** чувствительны к позиции рекомендаций в списке, что важно в данной задаче, поскольку пользователь с большей вероятностью купит товар, который находится в начале списка рекомендаций. А вот метрики **Precision** и **Recall** не учитывают позицию рекомендаций в списке, что может привести к тому, что модель будет рекомендовать товары, которые пользователь не купит.

Если сравнивать метрики **MAP@k** и **NDCG@k**, то послядняя основывается на градуированной оценке (с максимумом и минимумом), которой не имеется.  
Исходя из сказанного для оценки моделей можно использовать **MAP@6**, **MAP@12**.

$$
\text{AP} = \sum_n (R_n - R_{n-1}) P_n
$$

где $R_n$ и $P_n$ — точность и полнота на $n$-м пороге.  
**AP** можно сформулировать как поиск области под кривой **precision-recall**.
**MAP** — это среднее значение **AP** по всем запросам.

## <a id="property-solution"> </a>$\text{ Разработка baseline}$

Созадние разреженной матрици 

In [13]:
def long_to_wide(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns(
        interacted=pl.lit(1),  # Set all interactions to 1 initially
    )
    df.with_columns(pl.col("interacted").cast(pl.Int8))

    pivot = df.pivot(index='user_id', columns='item_id', values='interacted', aggregate_function="sum")
    return pivot

def wide_to_csr(df: pl.DataFrame) -> csr_matrix:
    arrow_array = df.to_arrow()
    values = arrow_array.to_pandas().values
    return csr_matrix(values)

def long_to_csr(df: pl.DataFrame) -> (csr_matrix, (np.array, np.array)):
    # Get unique user and item IDs
    users = df['user_id'].unique().to_list()
    items = df['item_id'].unique().to_list()

    # Create mapping from user/item ID to row/column index
    user_to_row = {user: i for i, user in enumerate(users)}
    item_to_col = {item: i for i, item in enumerate(items)}

    # Create data, row indices, and column indices for the CSR matrix
    data = [1] * len(df)
    row_indices = [user_to_row[user] for user in df['user_id']]
    col_indices = [item_to_col[item] for item in df['item_id']]

    # Create the CSR matrix
    csr = csr_matrix((data, (row_indices, col_indices)), 
                     shape=(len(users), len(items)),
                     dtype=np.int8)

    return csr, (np.array(users).flatten(), np.array(items).flatten())

### Test long_to_wide, wide_to_csr functions

In [14]:
wide = long_to_wide(test_rerank.head(10000))
csr = wide_to_csr(wide)

df = pl.DataFrame(csr.toarray(),)
df.columns=wide.columns
df = df.fill_nan(None)

assert_frame_equal(wide, df,check_dtype=False)

/tmp/ipykernel_35/447971455.py:7: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivot = df.pivot(index='user_id', columns='item_id', values='interacted', aggregate_function="sum")
/tmp/ipykernel_35/1402822435.py:8: DeprecationWarning: The argument `check_dtype` for `assert_frame_equal` is deprecated. It has been renamed to `check_dtypes`.
  assert_frame_equal(wide, df,check_dtype=False)


## Sparse matrix row to bought items

In [15]:
def user_purchases_from_sparse_optimized(sparse_data: csr_matrix, item_ids: np.array, user_index: int, return_index=False) -> Tuple[np.array, np.array]:
    user_interactions = sparse_data.getrow(user_index)
    user_interactions_dense = user_interactions.toarray().flatten()
    indices = np.where(user_interactions_dense != 0)[0]
    n_purchases = user_interactions_dense[indices]
    if return_index:
        return indices, n_purchases
    
    item_ids_of_user = item_ids[indices]
    return item_ids_of_user, n_purchases

def all_user_purchases_from_sparse(sparse_data: csr_matrix, item_ids: np.array,):
    all_user_purchases = []
    for user_index in range(sparse_data.shape[0]):
        item_ids_of_user, n_purchases = user_purchases_from_sparse_optimized(sparse_data, item_ids, user_index)
        all_user_purchases.append(item_ids_of_user)
    return all_user_purchases    

# Core Model Class

In [17]:
class BaseModel(ABC):
    @abstractmethod
    def fit(self, sparse_data: csr_matrix, item_ids: np.array, user_ids: np.array) -> 'BaseModel':
        pass
    
    @abstractmethod
    def predict(self, user_ids: np.array) -> np.array:
        pass
    # эти методы для послед. оптимизации параметров
    def _create_user_ids_to_user_index_map(self, user_ids: Iterable):
        return self._ids_to_index_map(user_ids)
    
    def _create_item_ids_to_item_index_map(self, item_ids: Iterable):
        return self._ids_to_index_map(item_ids)
    
    def _ids_to_index_map(self, ids: Iterable):
        n_ids = len(ids)
        return dict( zip(ids, range(n_ids)) )

## Sparse Dataset Palette

In [18]:
train_val_base_csr, (tv_user_ids, tv_item_ids) = long_to_csr(train_val_base)
cand_pred_csr, (cp_user_ids, cp_item_ids) = long_to_csr(candidate_predict_set)
debug_set_csr, (user_ids, item_ids) = long_to_csr(debug_set)

# Baseline

In [19]:
debug_set_coo = debug_set_csr.tocoo()

In [52]:
user_ids

array([     42,      44,      55, ..., 1057241, 1057245, 1057248])

## Popularity

In [21]:
class PopModel(BaseModel):
    def __init__(self):
        self.ranks = None
        self.n_items = None
    
    def fit(self, sparse_data: csr_matrix, item_ids: np.array, user_ids: np.array) -> PopModel:
        _ = user_ids
        assert sparse_data.shape == ( len(user_ids), len(item_ids) )
        self.n_items = len(item_ids)
        
        total_purchases = sparse_data.sum(axis=0)
        popularity_sorted_args = total_purchases.argsort()[:, ::-1]
        popularity_sorted_index = np.squeeze(np.asarray(popularity_sorted_args))
        total_purchases = np.squeeze(np.asarray(total_purchases))

        self.ranks = n_purchases = total_purchases[popularity_sorted_index]
        
        return self

    def predict(self, user_ids: np.array | Iterable) -> np.ndarray:
    
        n_users = len(user_ids)
        n_items = self.n_items
        
        ranks = self.ranks
        ranks_repeated = np.tile(ranks, (n_users, 1))
        
        assert ranks_repeated.shape == (n_users, n_items)
        return ranks_repeated

In [23]:
# pop_model = PopModel().fit(train_base_csr)
# pop_items = pop_model.predict(tb_item_ids)

pop_model = PopModel().fit(debug_set_csr, item_ids, user_ids)
pop_items = pop_model.predict(user_ids)
print("Popularity ordered, descending: ", pop_items)

print("\n1st, 2nd, last:")
print(train_base.filter(pl.col("item_id") == 192).shape)
print(train_base.filter(pl.col("item_id") == 390).shape)
print(train_base.filter(pl.col("item_id") == 8384).shape)

del pop_model

Popularity ordered, descending:  [[7240 5350 5131 ...    1    1    1]
 [7240 5350 5131 ...    1    1    1]
 [7240 5350 5131 ...    1    1    1]
 ...
 [7240 5350 5131 ...    1    1    1]
 [7240 5350 5131 ...    1    1    1]
 [7240 5350 5131 ...    1    1    1]]

1st, 2nd, last:
(244703, 3)
(179514, 3)
(1, 3)


## Historical

In [22]:
class UserHistoryModel(BaseModel):
    def __init__(self, chunk_size=10000):
        self.train_data = None
        self.user_ids_to_index = {}
        self.chunk_size = chunk_size  # Adjust the chunk size based on available memory and performance

        
    def fit(self, sparse_data: csr_matrix, item_ids: np.array, user_ids: np.array) -> UserHistoryModel:
      
        assert sparse_data.shape == ( len(user_ids), len(item_ids) )
        # it is itself ranks matrix
        self.train_data = sparse_data
        self.user_ids_to_index = self._create_user_ids_to_user_index_map(user_ids)
        return self

        
    def predict(self, user_ids: np.array) -> np.ndarray:
       
        user_indices = [self.user_ids_to_index.get(id_) for id_ in user_ids]
        user_indices = np.array(user_indices)

        # Deal with Cold Start, Just return 0 vectors
        mask = user_indices != None
        user_data = np.zeros(shape=(len(user_indices), self.train_data.shape[1]))  # Initialize with zeros
        
        logger.debug("Shapes: \n")
        logger.debug(f"mask.shape: {mask.shape}")
        logger.debug(f"user_data.shape: {user_data.shape}")
        logger.debug(f"self.train_data.shape: {self.train_data.shape}")
        logger.debug(f"user_indices.shape: {user_indices.shape}")
        
        chunk_size = self.chunk_size
        num_chunks = len(user_indices) // chunk_size + 1

        for i in range(num_chunks):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, len(user_indices))
            
            user_indices_chunk = user_indices[mask][start_idx:end_idx]
            user_data_chunk = self.train_data[user_indices_chunk, :].toarray()

            if len(user_indices_chunk) < end_idx - start_idx:
                end_idx = start_idx + len(user_indices_chunk)
                
            mask_chunk = mask[start_idx:end_idx]

            if user_data_chunk.shape[0] != mask_chunk.shape[0]:
                logger.debug(f"user_data_chunk.shape: {user_data_chunk.shape}, mask_chunk.shape: {mask_chunk.shape}")
            
            user_data[start_idx:end_idx, :][mask_chunk] = user_data_chunk[mask_chunk]

        return user_data

In [22]:
historical_model = UserHistoryModel().fit(debug_set_csr, np.array(item_ids), np.array(user_ids))

hist_predictions_ranks = historical_model.predict(user_ids)
hist_predictions_ranks[hist_predictions_ranks > 0]

del historical_model

2024-09-01 16:28:29.451 | DEBUG    | __main__:predict:30 - Shapes: 

2024-09-01 16:28:29.452 | DEBUG    | __main__:predict:31 - mask.shape: (118989,)
2024-09-01 16:28:29.453 | DEBUG    | __main__:predict:32 - user_data.shape: (118989, 4367)
2024-09-01 16:28:29.454 | DEBUG    | __main__:predict:33 - self.train_data.shape: (118989, 4367)
2024-09-01 16:28:29.455 | DEBUG    | __main__:predict:34 - user_indices.shape: (118989,)


## LightFM

In [23]:
!pip install lightfm

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=447717 sha256=0edd5a3e9c9acf3ad876fabd6c7be88c70802298075fd5fc56fc484669e50a0e
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [24]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k as lightfm_precision_at_k

class MatrixFactorization(BaseModel):
    def __init__(self, n_latent=10, model_class=LightFM, precision=lightfm_precision_at_k) -> MatrixFactorization:
        # Используем warp так как это лучший лосс для кейсов, когда только положительные взаимодействия в датасете
        self.model = model_class(loss="warp", no_components=n_latent)
        self.precision = lightfm_precision_at_k
        self.item_ids_fitted = None
        # num_threads=4 потому что у Kaggle 4 ядер.
        self.num_threads=4
    
    def fit(self, sparse_data: coo_matrix, item_ids: np.array, user_ids: np.array, epochs=1) -> MatrixFactorization:
        assert sparse_data.shape == ( len(user_ids), len(item_ids) )
        
        self.user_id_to_index = self._create_user_ids_to_user_index_map(user_ids)
        self.item_id_to_index = self._create_item_ids_to_item_index_map(item_ids)
        self.item_ids_fitted = item_ids

        %time self.model.fit(sparse_data, epochs=epochs, num_threads=self.num_threads)
        return self
    
    def predict(self, user_ids: np.array) -> np.ndarray:
        """
        Returns np.ndarray of shape N_USERS X N_ITEMS
        """
        # Lets set 0 as unknown id (Cold Start)
        user_indices = np.array([self.user_id_to_index.get(uid, 0) for uid in user_ids])
        item_indices = np.array([self.item_id_to_index.get(iid, 0) for iid in self.item_ids_fitted])
        
        n_users = len(user_indices)
        n_items = len(item_indices)

        repeated_user_indices = np.tile(user_indices.reshape(-1, 1), (1, n_items))
        repeated_user_indices = repeated_user_indices.astype(int)
        predictions = np.empty((n_users, n_items))
        
        for u in range(n_users):
        
            user_indices_repeated = repeated_user_indices[u, :]
            ranks_predicted = self.model.predict(user_indices_repeated, item_indices)
            predictions[u, :] = ranks_predicted
            
        return predictions
    
    def get_latent_users(self):
        return self.model.user_embeddings 
    
    def evaluate(self, test_data: csr_matrix, k=3):
        prec = self.precision(self.model, test_data, k=k)
        print(f"Train precision at {k}: %.2f" % prec.mean())
        return prec

In [25]:
mf_model = MatrixFactorization().fit(debug_set_csr, item_ids, user_ids, epochs=1)
test_preds = mf_model.predict(user_ids[:10])
mf_model.evaluate(debug_set_coo)
user_emb = mf_model.get_latent_users()
print(user_emb[:3])

del mf_model

CPU times: user 369 ms, sys: 6.69 ms, total: 375 ms
Wall time: 121 ms
Train precision at 3: 0.04
[[-0.02533926 -0.00761719  0.02113774 -0.09783773  0.06808431  0.0543194
   0.01429997 -0.06905469  0.00795502 -0.00904036]
 [ 0.0549169   0.047244    0.08424345  0.00278431  0.00921935  0.05296496
   0.00749297  0.08505103  0.03070669 -0.02176493]
 [ 0.10119444  0.08729941  0.02276123 -0.01392505  0.06407946 -0.05095175
   0.01711844 -0.06358068 -0.01316739  0.05694703]]


In [59]:
user_emb.shape

(118989, 10)

## User-to-User

In [26]:
# as NN already uses Approximate Nearest Neighbour, we will utilize this
from sklearn.neighbors import NearestNeighbors

class User2User(BaseModel):
    def __init__(self, metric="cosine", n_jobs=-1, n_neighbors=2):
        self.model = NearestNeighbors(n_neighbors=n_neighbors, radius=1, metric=metric, n_jobs=n_jobs)
        self.sparse_data_fitted = None
        self.item_ids = None
        self.user_ids = None
        self.user_id_to_index = {}
        self.user_embeddings = None
        
    def fit(self, sparse_data: csr_matrix, item_ids: Iterable, user_ids: Iterable, user_embeddings: np.ndarray):
        assert sparse_data.shape == ( len(user_ids), len(item_ids) )
        
        self.sparse_data_fitted = sparse_data
        self.item_ids = item_ids
        self.user_ids = user_ids
        self.user_id_to_index = self._create_user_ids_to_user_index_map(user_ids)
        
        # add zero row for unknown users (cold start)
        zero_row = np.zeros((1, user_embeddings.shape[1]))
        self.user_embeddings = np.vstack((user_embeddings, zero_row))
        
        logger.info(f"{user_embeddings.shape[0]} users to estimate left...")
        self.model.fit(user_embeddings)
        logger.info(f"Done.")
        return self
    
    def predict(self, user_ids: Iterable):
        user_indices = np.array([self.user_id_to_index.get(uid, -1) for uid in user_ids])
        user_embeddings = self.user_embeddings[user_indices]

        dist, similar_users_index_matrix = self.model.kneighbors(user_embeddings, return_distance=True)
        
        all_users_ranks = np.zeros((len(user_ids), len(self.item_ids)))
        rank_positions = 1 / (np.arange(len(similar_users_index_matrix[0])) + 1)
        
        for i, similar_users in enumerate(similar_users_index_matrix):
            if i % 100000 == 0:
                logger.debug(f"{i} Users processed out of {similar_users_index_matrix.shape[0]}")
            ranks_for_user = self._find_and_rank_neighbors(similar_users, rank_positions)
            all_users_ranks[i, :] = ranks_for_user
        
        return all_users_ranks

    def _find_and_rank_neighbors(self, similar_users: np.array, rank_positions: np.array):
        ranks_for_users = np.zeros((len(similar_users), len(self.item_ids)))
        
        for i, similar_user_index in enumerate(similar_users):
            similar_user_item_indices, n_purchases = self._get_user_items_and_ranks(similar_user_index)
            ranks_for_users[i, similar_user_item_indices] = n_purchases * rank_positions[i]
        
        return np.sum(ranks_for_users, axis=0)

    def _get_user_items_and_ranks(self, user_index: int):
        similar_user_item_indices, n_purchases = user_purchases_from_sparse_optimized(self.sparse_data_fitted, self.item_ids, user_index, return_index=True)
        return similar_user_item_indices, n_purchases.flatten()

# Wall time 3:40
%time knn_model = User2User().fit(debug_set_csr, item_ids, user_ids, user_emb)
%time ranks = knn_model.predict(user_ids[:10])
del knn_model, ranks

2024-09-01 16:47:22.104 | INFO     | __main__:fit:25 - 118989 users to estimate left...
2024-09-01 16:47:22.108 | INFO     | __main__:fit:27 - Done.
2024-09-01 16:47:22.170 | DEBUG    | __main__:predict:41 - 0 Users processed out of 10


CPU times: user 34.3 ms, sys: 4.02 ms, total: 38.3 ms
Wall time: 38.6 ms
CPU times: user 120 ms, sys: 126 ms, total: 246 ms
Wall time: 70.6 ms


Все сходится, рекомендуются резонно:

In [61]:
for r in ranks[:3]:
    print(item_ids[np.where(r > 0)])

NameError: name 'ranks' is not defined

In [ ]:
# debug_set_csr[0,:]
for item_index in range(3):
    print(user_purchases_from_sparse_optimized(debug_set_csr, item_ids, item_index) )

(array([297]), array([1], dtype=int8))

(array([1855]), array([1], dtype=int8))

(array([ 133,  212, 5517]), array([1, 1, 1], dtype=int8))


## Sequential

In [28]:
config_dict = {
    'single_spec': True,
    'NEG_PREFIX': 'neg_',
    "USER_ID_FIELD": "user_id",
    "ITEM_ID_FIELD": "item_id",
    "TIME_FIELD": "timestamp",
    "load_col": {"inter": ["user_id", "item_id", "timestamp"]},
    "ITEM_LIST_LENGTH_FIELD": "item_length",
    "LIST_SUFFIX": "_list",
    "MAX_ITEM_LIST_LENGTH": 20,  # максимальная длина последовательности
    "embedding_size": 256, # размерность эмбеддингов
    "hidden_size": 256, # размерность скрытого состояния GRU
    "num_layers": 1,  # кол-во скрытых слоёв
    "dropout_prob": 0.3,  # droupout-вероятность в обучении
    "loss_type": "CE",  # функция потерь
    "epochs": 50,
    "train_batch_size": 256,
    "eval_batch_size": 256,
    "train_neg_sample_args": None, # не используем negative sampling
    # словарь с параметрами валидации
    "eval_args": {
        "group_by": "user", # группируем по пользователям
        "order": "TO", # сортируем по timestamp-у
        "split": {"LS": "valid_only"}, # leave-one-out validation
        "mode": "full", # используем все данные для inference
    },
    "metrics": ["Recall", "MRR", "NDCG", "Hit", "Precision", "MAP"],
    "topk": 10,
    "valid_metric": "MAP@10", # валидационная метрика, также целевая метрика
    # соревнования
    "data_path": "alfa_data/",
    "dataset": "full_train",
    "stopping_step": 2, # останавливаем обучение если в течение
    "save_interval": 1,
    "save_final_model": True,
    "device": DEVICE,
}

In [29]:
!pip install kmeans_pytorch

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [30]:
class SequentialGRU4Rec(BaseModel):
    
    def __init__(self,  config_dict): 
#         self.data = data
        self.config_dict = config_dict 
        self.train_data = None
        self.valid_data = None  
        self.configg = None
        self.model = None 
        
    def data_process(self, data):
        # Переименование столбцов
            # Переименование столбцов
        data = data.rename({'user_id': 'user_id:token', 'item_id': 'item_id:token', 'order_ts': 'timestamp:float'})

        # Преобразование строки в datetime с учетом микро-секунд
        data = data.with_columns(
            pl.col("timestamp:float").cast(pl.Utf8).str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S.%f")
        )

        # Преобразование datetime в Unix timestamp в микросекундах
        data = data.with_columns(
            pl.col("timestamp:float").dt.timestamp().alias("timestamp:float")
        )


        print(data)

        # Сохранение данных
        self.data = data
        new_folder = 'alfa_data/full_train'

        # Создаём новую папку 'full_train' если она не существует
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)

        new_csv = 'alfa_data/full_train/full_train.inter'

        # Сохраняем csv с данными для обучения если ещё этого не делали
        if not os.path.exists(new_csv):
            data = data.select(['user_id:token', 'item_id:token', 'timestamp:float'])
            data.write_csv(new_csv, separator="\t")
        
        
    def get_data_split(self):
        self.configg = Config(model='GRU4Rec', dataset='full_train', config_dict=self.config_dict)

        # инициализируем логгеры для вывода информации
        logger = getLogger()
        logger.setLevel(logging.INFO)


        c_handler = logging.StreamHandler()
        c_handler.setLevel(logging.INFO)
        logger.addHandler(c_handler)


        logger.info(self.configg)
        
        dataset = create_dataset(self.configg)
        logger.info(dataset)
        self.train_data, self.valid_data, _ = data_preparation(self.configg, dataset)
#         print(result)
        
        
    def fit(self, data):
        self.data_process(data)
        self.get_data_split()
        
        self.model = GRU4Rec(self.configg, self.train_data.dataset).to(self.configg['device'])
        logger.info(self.model)

        # инициализируем "тренера" модели
        trainer = Trainer(self.configg, self.model)

        # сохраняем лучшие результаты
        best_valid_score, best_valid_result = trainer.fit(self.train_data, self.valid_data)

    def predict(self, k=1):
            num_users = self.train_data.dataset.user_num
            num_items = self.train_data.dataset.item_num

            topk_items = []

            # Ensure the model has been trained
            if self.train_data is None or self.valid_data is None or self.configg is None or not hasattr(self, 'model'):
                raise ValueError("Модель еще не была обучена. Пожалуйста, вызовите метод `fit` перед использованием `predict`.")

            # Initialize a 2D array for predictions
            predictions = np.zeros((num_users - 1, num_items - 1))

            # Loop over all users in the dataset
            for internal_user_id in range(num_users):
                _, topk_iid_list = full_sort_topk(
                    [internal_user_id],
                    self.model,
                    self.valid_data,
                    k=k,
                    device=self.configg['device']
                )

                if len(topk_iid_list) > 0:
                    last_topk_iid_list = topk_iid_list[-1]

                    # Create a ranking score array
                    for rank, item_id in enumerate(last_topk_iid_list):
                        predictions[internal_user_id, item_id] = 1 / (rank + 1)  # or any other ranking score
                else:
                    # If no recommendations, leave the array as zeros
                    pass

            return predictions




In [ ]:
# sec = SequentialGRU4Rec(config_dict)
# sec.fit(data)  
# recommendations = sec.predict(k=15)

не получилось привесть обучение модели последовательных рекомендаций к единому интерфейсу. 

## Model Ensemble

In [32]:
class ModelEnsemble:
    def __init__(self, config_dict):
        self.pop_model = PopModel()
        self.history_model = UserHistoryModel()
        self.mf_model = MatrixFactorization()
        self.knn_model = User2User(n_neighbors=2)
        self.sec_model = SequentialGRU4Rec(config_dict)
        
        self.models = {
            "Popularity Based Model": self.pop_model,
            "User History Model": self.history_model,
            "Matrix Factorization": self.mf_model,
            "User2User": self.knn_model,
            "Sequential GRU4Rec": self.sec_model  
        }
        
    def fit(self, data: coo_matrix, user_ids: Iterable, item_ids: Iterable, train) -> 'ModelEnsemble':
        simple_fit_models = [(k, v) for k, v in self.models.items() if k != "Sequential GRU4Rec" and k != "User2User"]
        assert data.shape == (len(user_ids), len(item_ids))

        for name, model in simple_fit_models:
            logger.info(f"Training model {name}")
            model.fit(data, user_ids=user_ids, item_ids=item_ids)
        
        user_embeddings = self.mf_model.get_latent_users()
        logger.info(f"Training model User2User")
        self.knn_model.fit(data, item_ids, user_ids, user_embeddings)
    
        logger.info(f"Training Sequential GRU4Rec model")

        self.sec_model.fit(train)
        
        return self
        
    def predict(self, user_ids: Iterable) -> Dict[str, np.ndarray]:
        predictions = {model_name: None for model_name in self.models.keys()}
        
        for name, model in self.models.items():
            logger.info(f"Inferencing model {name}")
            if name == "Sequential GRU4Rec":

                ranks_pred = model.predict(k=1)
            else:
                ranks_pred = model.predict(user_ids)
            
            predictions[name] =  ranks_pred
        return predictions
    
    def __del__(self):
        for name, model in self.models.items():
            del model


In [30]:
ensemble = ModelEnsemble(config_dict)

In [31]:
ensemble.fit(debug_set_csr, user_ids, item_ids, debug_set)

2024-09-01 11:08:32.432 | INFO     | __main__:fit:22 - Training model Popularity Based Model
2024-09-01 11:08:32.439 | INFO     | __main__:fit:22 - Training model User History Model
2024-09-01 11:08:32.481 | INFO     | __main__:fit:22 - Training model Matrix Factorization
2024-09-01 11:08:32.636 | INFO     | __main__:fit:26 - Training model User2User
2024-09-01 11:08:32.684 | INFO     | __main__:fit:25 - 118989 users to estimate left...
2024-09-01 11:08:32.687 | INFO     | __main__:fit:27 - Done.
2024-09-01 11:08:32.688 | INFO     | __main__:fit:29 - Training Sequential GRU4Rec model


CPU times: user 360 ms, sys: 5.07 ms, total: 365 ms
Wall time: 115 ms


/tmp/ipykernel_36/2521008182.py:18: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("timestamp:float").cast(pl.Utf8).str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S.%f")

General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = alfa_data/full_train
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 256
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 2
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

shape: (272_593, 3)
┌───────────────┬───────────────┬──────────────────┐
│ user_id:token ┆ item_id:token ┆ timestamp:float  │
│ ---           ┆ ---           ┆ ---              │
│ i64           ┆ i64           ┆ i64              │
╞═══════════════╪═══════════════╪══════════════════╡
│ 550614        ┆ 264           ┆ 1672532889000000 │
│ 571051        ┆ 580           ┆ 1672533707000000 │
│ 571051        ┆ 180           ┆ 1672533707000000 │
│ 47164         ┆ 5135          ┆ 1672534415000000 │
│ 219072        ┆ 2668          ┆ 1672534949000000 │
│ …             ┆ …             ┆ …                │
│ 491658        ┆ 1202          ┆ 1672702634000000 │
│ 758413        ┆ 1268          ┆ 1672703166000000 │
│ 841619        ┆ 2780          ┆ 1672703237000000 │
│ 92811         ┆ 5779          ┆ 1672703512000000 │
│ 55287         ┆ 6315          ┆ 1672703696000000 │
└───────────────┴───────────────┴──────────────────┘


full_train
The number of users: 118990
Average actions of users: 2.2909092437116034
The number of items: 4368
Average actions of items: 62.421112892145636
The number of inters: 272593
The sparsity of the dataset: 99.94755288482703%
Remain Fields: ['user_id', 'item_id', 'timestamp']
[Training]: train_batch_size = [256] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Evaluation]: eval_batch_size = [256] eval_args: [{'split': {'LS': 'valid_only'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]
2024-09-01 11:08:40.756 | INFO     | __main__:fit:71 - GRU4Rec(
  (item_embedding): Embedding(4368, 256, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(256, 256, bias=False, batch_first=True)
  (dense): Linear(in_features=256, out_features=256, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 1577216
epoch 0 training [time: 6.95s, train

In [32]:
preds = ensemble.predict(user_ids)
list(preds.keys())


prediction_shapes = []
for name, prediction_of_model in preds.items():
    shape = prediction_of_model.shape
    prediction_shapes.append(shape)
    print(name,":", shape)

assert prediction_shapes.count(prediction_shapes[0]) == len(prediction_shapes), "Some models have wrong shape"

2024-09-01 11:10:43.213 | INFO     | __main__:predict:39 - Inferencing model Popularity Based Model
2024-09-01 11:10:44.127 | INFO     | __main__:predict:39 - Inferencing model User History Model
2024-09-01 11:10:44.189 | DEBUG    | __main__:predict:30 - Shapes: 

2024-09-01 11:10:44.190 | DEBUG    | __main__:predict:31 - mask.shape: (118989,)
2024-09-01 11:10:44.191 | DEBUG    | __main__:predict:32 - user_data.shape: (118989, 4367)
2024-09-01 11:10:44.191 | DEBUG    | __main__:predict:33 - self.train_data.shape: (118989, 4367)
2024-09-01 11:10:44.192 | DEBUG    | __main__:predict:34 - user_indices.shape: (118989,)
2024-09-01 11:10:47.292 | INFO     | __main__:predict:39 - Inferencing model Matrix Factorization
2024-09-01 11:11:56.266 | INFO     | __main__:predict:39 - Inferencing model User2User
2024-09-01 11:15:58.591 | DEBUG    | __main__:predict:41 - 0 Users processed out of 118989
2024-09-01 11:16:18.686 | DEBUG    | __main__:predict:41 - 100000 Users processed out of 118989
2024-

Popularity Based Model : (118989, 4367)
User History Model : (118989, 4367)
Matrix Factorization : (118989, 4367)
User2User : (118989, 4367)
Sequential GRU4Rec : (118989, 4367)


In [41]:
del preds
gc.collect()

NameError: name 'preds' is not defined

### Final Ensemble

In [34]:
# Training on full 1-stage dataset
try: 
    del ensemble
except:
    pass

In [36]:
import gc
gc.collect()

252

In [3]:
import torch

In [ ]:
ensemble = ModelEnsemble(config_dict).fit(train_val_base_csr, tv_user_ids, tv_item_ids, train_val_base)

2024-09-01 12:35:09.787 | INFO     | __main__:fit:22 - Training model Popularity Based Model
2024-09-01 12:35:09.841 | INFO     | __main__:fit:22 - Training model User History Model
2024-09-01 12:35:10.095 | INFO     | __main__:fit:22 - Training model Matrix Factorization
2024-09-01 12:35:15.333 | INFO     | __main__:fit:26 - Training model User2User


CPU times: user 17.5 s, sys: 109 ms, total: 17.6 s
Wall time: 4.98 s


2024-09-01 12:35:15.645 | INFO     | __main__:fit:25 - 941415 users to estimate left...
2024-09-01 12:35:15.652 | INFO     | __main__:fit:27 - Done.
2024-09-01 12:35:15.653 | INFO     | __main__:fit:29 - Training Sequential GRU4Rec model
/tmp/ipykernel_36/2521008182.py:18: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("timestamp:float").cast(pl.Utf8).str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S.%f")

General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = alfa_data/full_train
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 256
learner = adam
learn

shape: (13_339_315, 3)
┌───────────────┬───────────────┬──────────────────┐
│ user_id:token ┆ item_id:token ┆ timestamp:float  │
│ ---           ┆ ---           ┆ ---              │
│ i64           ┆ i64           ┆ i64              │
╞═══════════════╪═══════════════╪══════════════════╡
│ 550614        ┆ 264           ┆ 1672532889000000 │
│ 571051        ┆ 580           ┆ 1672533707000000 │
│ 571051        ┆ 180           ┆ 1672533707000000 │
│ 47164         ┆ 5135          ┆ 1672534415000000 │
│ 219072        ┆ 2668          ┆ 1672534949000000 │
│ …             ┆ …             ┆ …                │
│ 268063        ┆ 69            ┆ 1677628452000687 │
│ 340866        ┆ 212           ┆ 1677628629000836 │
│ 230960        ┆ 41            ┆ 1677628634000883 │
│ 86393         ┆ 148           ┆ 1677628640000498 │
│ 295015        ┆ 131           ┆ 1677628693000119 │
└───────────────┴───────────────┴──────────────────┘


full_train
The number of users: 941416
Average actions of users: 14.169431122299942
The number of items: 6416
Average actions of items: 2079.394388152767
The number of inters: 13339315
The sparsity of the dataset: 99.77915498642285%
Remain Fields: ['user_id', 'item_id', 'timestamp']
full_train
The number of users: 941416
Average actions of users: 14.169431122299942
The number of items: 6416
Average actions of items: 2079.394388152767
The number of inters: 13339315
The sparsity of the dataset: 99.77915498642285%
Remain Fields: ['user_id', 'item_id', 'timestamp']
[Training]: train_batch_size = [256] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [256] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Evaluation]: eval_batch_size = [256] eval_args: [{'split': {'LS': 'valid_only'}, 'order': 'TO', 'group_

In [33]:
ensemble = ModelEnsemble(config_dict).fit(train_val_base_csr, tv_user_ids, tv_item_ids, train_val_base)

2024-09-01 16:48:26.124 | INFO     | __main__:fit:22 - Training model Popularity Based Model
2024-09-01 16:48:26.181 | INFO     | __main__:fit:22 - Training model User History Model
2024-09-01 16:48:26.468 | INFO     | __main__:fit:22 - Training model Matrix Factorization
2024-09-01 16:48:31.499 | INFO     | __main__:fit:26 - Training model User2User


CPU times: user 16.7 s, sys: 108 ms, total: 16.8 s
Wall time: 4.77 s


2024-09-01 16:48:31.823 | INFO     | __main__:fit:25 - 941415 users to estimate left...
2024-09-01 16:48:31.829 | INFO     | __main__:fit:27 - Done.
2024-09-01 16:48:31.830 | INFO     | __main__:fit:29 - Training Sequential GRU4Rec model
/tmp/ipykernel_35/3376280111.py:18: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("timestamp:float").cast(pl.Utf8).str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S.%f")


shape: (13_339_315, 3)
┌───────────────┬───────────────┬──────────────────┐
│ user_id:token ┆ item_id:token ┆ timestamp:float  │
│ ---           ┆ ---           ┆ ---              │
│ i64           ┆ i64           ┆ i64              │
╞═══════════════╪═══════════════╪══════════════════╡
│ 550614        ┆ 264           ┆ 1672532889000000 │
│ 571051        ┆ 580           ┆ 1672533707000000 │
│ 571051        ┆ 180           ┆ 1672533707000000 │
│ 47164         ┆ 5135          ┆ 1672534415000000 │
│ 219072        ┆ 2668          ┆ 1672534949000000 │
│ …             ┆ …             ┆ …                │
│ 268063        ┆ 69            ┆ 1677628452000687 │
│ 340866        ┆ 212           ┆ 1677628629000836 │
│ 230960        ┆ 41            ┆ 1677628634000883 │
│ 86393         ┆ 148           ┆ 1677628640000498 │
│ 295015        ┆ 131           ┆ 1677628693000119 │
└───────────────┴───────────────┴──────────────────┘



General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = alfa_data/full_train
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 256
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 2
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_only'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk = [10]
valid_metric = MAP@10
valid_metric_bigger = True
eval_batch_size = 256
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separato

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">  

Оба последних обучения производильсь на *kaggle* . Првая модель обученна на GPU T4 x 2, воторая на GPU P100. Последняя смогла обучить модель быстрее. 

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">   
    
    
Модель **GRU4Rec** обучается долго. Однако обучающую выборку можно уменьшить путем удаления из нее неактивных пользователей. Это не должно сильно сказаться на качестве модели, поскольку такие пользователи не несут информацию о последовательностях.   
    
А также возможен подход реализующий разделение обучающего набора данных на части. Разделение даст возможность обучить несколько простых моделей, предсказание от которых можно совместить.  
    
*Стоит отметить, что модель последовательных рекомендаций уже на первых итерация достигает хороших результатов, но дальше обучается медленно*

In [43]:
import gc
gc.collect()

0

In [3]:
# cand_pred_csr, (cp_user_ids, cp_item_ids)
fraction_size = int(len(cp_user_ids) * 0.05)
cp_user_ids_frac = np.random.choice(cp_user_ids, size=fraction_size)
candidate_predictions = ensemble.predict(cp_user_ids_frac)

NameError: name 'cp_user_ids' is not defined

In [43]:
for model, ranks in candidate_predictions.items():
    np.save(f'{model}_ranks.npy', ranks)

In [ ]:
!ls

'=0.4.4'			    'User History Model_ranks.npy'

'Matrix Factorization_ranks.npy'     User2User_ranks.npy

'Popularity Based Model_ranks.npy'


## <a id="choice-solution"> </a>$\text{ Реализация выбранных решиений }$ 

## Second Stage (CatBoost)

### Data Preprocessing

Как мы формируем выборку для catboost?
Тренируем на задачу классификации. На каждую пару user-item (+/-), должны быть ранги всех моделей по этой паре.

Изначально можно собрать все взаимодействия, которые правда были и собрать ранги из моделей. Благо каждый датасет с предсказаниями выдает
предсказания именно в таком порядке, в котором они были запрошены, следовательно простым индексированием row-column можно получить любые ранги
на любую пару пользователя и айтема для любой модели, это стоит организовать в функцию.

In [45]:
pop_ranks = candidate_predictions['Popularity Based Model']
hist_ranks = candidate_predictions['User History Model']
mf_ranks = candidate_predictions['Matrix Factorization']
knn_ranks = candidate_predictions['User2User']

all_ranks = [pop_ranks, hist_ranks, mf_ranks, knn_ranks]

In [66]:
user_ids, item_ids

(array([     42,      44,      55, ..., 1057241, 1057245, 1057248]),
 array([  11,   24,   27, ..., 8336, 8338, 8344]))

In [47]:
len(candidate_predictions["Sequential GRU4Rec"])

118989

In [ ]:
candidate_predictions['Matrix Factorization']

In [50]:
candidate_predictions['User2User']

array([[0.5, 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [1. , 0. , 0. , ..., 0. , 0. , 0. ]])

### get_positive_samples Function

In [67]:
def get_rank_for_pair_index(user_index: int, item_index: int, ranks: np.ndarray) -> float:
 
    rank = ranks[user_index, item_index]
    if type(rank) == np.matrix:
        rank = np.array(rank)
    elif type(rank) in (csr_matrix, coo_matrix):
        rank = rank.toarray()
    else: 
        assert type(rank) == np.ndarray
        
    if rank.size == 0:
        return None
    return float( rank.flatten()[0] )
    

def get_rank_for_pair_ids(user_id: int, item_id: int, ranks: np.ndarray, 
                         user_ids: Iterable, item_ids: Iterable):

    user_index = np.where(user_ids==user_id)[0]
    item_index = np.where(item_ids==item_id)[0]
    if user_index.size == 0 or item_index.size == 0:
        return None  # или любое другое значение по умолчанию, которое имеет смысл в вашем контексте
    
    return get_rank_for_pair_index(user_index, item_index, ranks)
    


In [68]:
def append_ranks(ranks: np.ndarray, df: pl.DataFrame, user_ids: Iterable, item_ids: Iterable, col_name: str) -> pl.DataFrame:
    user_col_index = 1
    item_col_index = 2
    
    rank_df = df.map_rows(
        lambda row: get_rank_for_pair_ids(row[user_col_index], row[item_col_index], ranks, user_ids, item_ids), 
        return_dtype=pl.Float32
   )
    return df.with_columns(rank_df.to_series().alias(col_name))

In [69]:
def get_positive_samples(df: pl.DataFrame, all_ranks: List[np.ndarray], user_ids: Iterable, item_ids, col_names: List[str]):
    """user_ids and item_ids should be taken from train, those have to be in ranks matrix as rows and columns"""
    df_known_users = df.filter(pl.col("user_id").is_in(user_ids))
    df_with_ranks = df_known_users
    
    for ranks, col_name in zip(all_ranks, col_names):
        df_with_ranks = append_ranks(ranks, df_with_ranks, user_ids, item_ids, col_name)
    df_with_ranks = df_with_ranks.with_columns(target=pl.lit(1))
    
    return df_with_ranks

In [70]:
some_users = user_ids[:5]
some_items = item_ids[:5]

for item_id, user_id in zip(some_items, some_users):
    some_ranks = [get_rank_for_pair_ids(user_id,item_id,ranks, user_ids, item_ids) for ranks in all_ranks]
    print(some_ranks)

[296936.0, 0.0, 2.9419963359832764, 0.0]

[242022.0, 0.0, 1.9102529287338257, 0.0]

[214386.0, 0.0, 0.6002499461174011, 0.0]

[199295.0, 0.0, 0.8810538649559021, 0.0]

[199056.0, 0.0, 0.20081593096256256, 0.0]


In [71]:
col_names = ["pop_rank", "hist_rank", "mf_rank", "knn_rank"]
debug_positive_with_ranks = get_positive_samples(debug_set, all_ranks, user_ids[:10], item_ids, col_names)
debug_positive_with_ranks.head()

,user_id,item_id,order_ts,pop_rank,hist_rank,mf_rank,knn_rank,target
i64,i64,i64,datetime[μs],f32,f32,f32,f32,i32
1548,44,1855,2023-01-01 09:53:17,1106.0,0.0,0.774501,0.0,1
13638,55,212,2023-01-02 19:29:19,21475.0,0.0,-0.945148,0.0,1
223210,88,7642,2023-01-01 12:49:39,44.0,0.0,-1.230841,0.0,1
690092,95,3313,2023-01-02 09:29:01,355.0,0.0,-1.914413,0.0,1
1385341,88,390,2023-01-01 12:49:39,10457.0,0.0,2.368595,0.0,1


### generate_negative_examples Function

In [72]:


def create_pos_memo(positive_df: pl.DataFrame, user_id: Iterable, item_id: Iterable):
    pos_user_ids = positive_df["item_id"]
    pos_item_ids = positive_df["user_id"]

    positive_examples_memo = {(user_id, item_id): True for user_id, item_id in zip(pos_user_ids, pos_item_ids)}
    return positive_examples_memo
    
def generate_negative_examples(user_ids: Iterable, item_ids: Iterable, 
                               positive_examples_memo: Dict[Tuple[int, int]: bool], 
                               all_ranks: List[np.ndarray], df_schema: SchemaDefinition, n=10):
    n_predicted_users = all_ranks[-1].shape[0]
        
#     assert n_predicted_users >= n, f"Cant generate so much samples: {n}, as predictions are presented only for {n_predicted_users}"
    
    user_ids = user_ids.copy()
    item_ids = item_ids.copy()
    shuffle(user_ids)
    shuffle(item_ids)

    n_items = item_ids.shape[0]
    negative_examples = np.empty( (n, 5 + len(all_ranks)) )
    j = 0
    
    while j < n:
        # if overflow occurs: n is bigger then number of users, then iterate through users again
        i = j % n_items
        if i == 0 or i % 25000 == 0:
            logger.info(f"{j} neg. samples generated our of {n}")
        uid, iid = user_ids[i], item_ids[i]

        if positive_examples_memo.get((uid, iid)) is True:
            n += 1
            logger.debug(f"{(uid, iid)} is found in positive samples, skipping")
            continue

        ranks_for_pair = []
        for ranks in all_ranks:
            rank = get_rank_for_pair_ids(uid, iid, ranks, user_ids, item_ids)
            ranks_for_pair.append(rank)

        # index, uid, iid, ts, [rank_1, ..., rank_n], target
        row = [0, uid, iid, 0] + ranks_for_pair + [0]
        negative_examples[i, :] = np.array(row)

        j += 1

    return pl.DataFrame(data=negative_examples, schema=df_schema, nan_to_null=True)

In [73]:
df_schema = debug_positive_with_ranks.schema
pos_memo = create_pos_memo(debug_positive_with_ranks, user_ids, item_ids)
debug_negatives_with_rank = generate_negative_examples(user_ids, item_ids, pos_memo, all_ranks, df_schema, n=10)
debug_negatives_with_rank.head()

2024-08-31 22:38:47.119 | INFO     | __main__:generate_negative_examples:28 - 0 neg. samples generated our of 10


,user_id,item_id,order_ts,pop_rank,hist_rank,mf_rank,knn_rank,target
i64,i64,i64,datetime[μs],f32,f32,f32,f32,i32
0,307144,3777,1970-01-01 00:00:00,296936.0,0.0,2.941996,0.0,0
0,26232,6143,1970-01-01 00:00:00,242022.0,0.0,1.910253,0.0,0
0,75260,7610,1970-01-01 00:00:00,214386.0,0.0,0.60025,0.0,0
0,270851,3242,1970-01-01 00:00:00,199295.0,0.0,0.881054,0.0,0
0,213528,3355,1970-01-01 00:00:00,199056.0,0.0,0.200816,0.0,0


### CatBoost Training

#### Prepare Data

In [74]:
all_ranks = list(candidate_predictions.values())
col_names = ["pop_rank", "hist_rank", "mf_rank", "knn_rank"]

pos_neg_split = 0.66

pos = get_positive_samples(train_val_base, all_ranks, cp_user_ids_frac, cp_item_ids, col_names)
pos_dict = create_pos_memo(pos, cp_user_ids_frac, cp_item_ids)

n_positives = pos.shape[0]
n_negatives = int((1 - pos_neg_split) * n_positives)

logger.debug(f"pos.shape: {pos.shape}")
logger.debug(f"{n_negatives} negative samples will be sampled, %{1 - pos_neg_split}")

2024-08-31 22:39:21.597 | DEBUG    | __main__:<module>:12 - pos.shape: (606088, 9)

2024-08-31 22:39:21.598 | DEBUG    | __main__:<module>:13 - 206069 negative samples will be sampled, %0.33999999999999997


In [75]:
neg = generate_negative_examples(cp_user_ids_frac, cp_item_ids, pos_dict, all_ranks, pos.schema, n=n_negatives)
logger.debug(f"neg.shape: {neg.shape}")

2024-08-31 22:40:06.175 | INFO     | __main__:generate_negative_examples:28 - 0 neg. samples generated our of 206069

2024-08-31 22:40:06.534 | INFO     | __main__:generate_negative_examples:28 - 6310 neg. samples generated our of 206069

2024-08-31 22:40:06.837 | INFO     | __main__:generate_negative_examples:28 - 12620 neg. samples generated our of 206069

2024-08-31 22:40:07.137 | INFO     | __main__:generate_negative_examples:28 - 18930 neg. samples generated our of 206069

2024-08-31 22:40:07.437 | INFO     | __main__:generate_negative_examples:28 - 25240 neg. samples generated our of 206069

2024-08-31 22:40:07.738 | INFO     | __main__:generate_negative_examples:28 - 31550 neg. samples generated our of 206069

2024-08-31 22:40:08.044 | INFO     | __main__:generate_negative_examples:28 - 37860 neg. samples generated our of 206069

2024-08-31 22:40:08.347 | INFO     | __main__:generate_negative_examples:28 - 44170 neg. samples generated our of 206069

2024-08-31 22:40:08.646 | INF

In [76]:
boost_data = pl.concat((pos, neg))
boost_data.sample(5)

,user_id,item_id,order_ts,pop_rank,hist_rank,mf_rank,knn_rank,target
i64,i64,i64,datetime[μs],f32,f32,f32,f32,i32
0,0,0,1970-01-01 00:00:00,0.0,0.0,0.0,0.0,0
10408071,451402,286,2023-02-13 10:55:08.500108,16083.0,0.0,1.981851,1.0,1
8131649,165026,1316,2023-02-02 14:12:28.592899,1774.0,0.0,-1.650276,0.0,1
2639398,1027636,3201,2023-01-08 11:33:24,261.0,0.0,-1.803313,0.0,1
0,0,0,1970-01-01 00:00:00,0.0,0.0,0.0,0.0,0


In [77]:
display(pos.sample(5))
display(neg.sample(5))

,user_id,item_id,order_ts,pop_rank,hist_rank,mf_rank,knn_rank,target
i64,i64,i64,datetime[μs],f32,f32,f32,f32,i32
3336945,817779,1007,2023-01-13 08:23:17,2769.0,0.0,-1.318078,0.0,1
6450976,1000478,1918,2023-01-27 14:52:32.848219,854.0,0.0,-0.593286,0.0,1
8228125,839046,201,2023-02-05 00:04:02.609296,22860.0,0.0,-0.053717,0.0,1
1802022,597146,163,2023-01-05 18:06:31,28957.0,0.0,2.263837,0.0,1
1464335,213679,2827,2023-01-01 11:40:13,372.0,0.0,-2.172124,0.0,1


,user_id,item_id,order_ts,pop_rank,hist_rank,mf_rank,knn_rank,target
i64,i64,i64,datetime[μs],f32,f32,f32,f32,i32
0,0,0,1970-01-01 00:00:00,0.0,0.0,0.0,0.0,0
0,0,0,1970-01-01 00:00:00,0.0,0.0,0.0,0.0,0
0,0,0,1970-01-01 00:00:00,0.0,0.0,0.0,0.0,0
0,0,0,1970-01-01 00:00:00,0.0,0.0,0.0,0.0,0
0,0,0,1970-01-01 00:00:00,0.0,0.0,0.0,0.0,0


In [78]:
ctb_train_users = train_rerank["user_id"].unique()
ctb_test_users =  test_rerank["user_id"].unique()

In [79]:
# 10% под механизм early stopping
ctb_train_users, ctb_eval_users  = train_test_split(ctb_train_users,
                                                  random_state=1,
                                                  test_size=0.1)

In [80]:
select_col = ['user_id', 'item_id', 'target'] + col_names

# Catboost train
ctb_train = pl.concat([
                pos.filter(pl.col('user_id').is_in(ctb_train_users) ),
                neg.filter(pl.col('user_id').is_in(ctb_train_users) )
])[select_col]
ctb_train = ctb_train.select(pl.col(select_col).shuffle(seed=1))


# Catboost test
ctb_test = pl.concat([
                pos.filter(pl.col('user_id').is_in(ctb_test_users) ),
                neg.filter(pl.col('user_id').is_in(ctb_test_users) )
])[select_col]
ctb_test = ctb_test.select(pl.col(select_col).shuffle(seed=1))


# for early stopping
ctb_eval = pl.concat([
                pos.filter(pl.col('user_id').is_in(ctb_eval_users) ),
                neg.filter(pl.col('user_id').is_in(ctb_eval_users) )
])[select_col]
ctb_eval = ctb_eval.select(pl.col(select_col).shuffle(seed=1))



In [ ]:
ctb_train

user_id,item_id,target,pop_rank,hist_rank,mf_rank,knn_rank
i64,i64,i32,f32,f32,f32,f32
692883,346,1,11382.0,0.0,1.080779,0.0
1023268,520,1,6810.0,0.0,1.532645,0.0
73189,1110,1,2319.0,0.0,0.712726,0.0
81744,1613,1,1153.0,0.0,-2.557565,0.0
54335,6207,1,14.0,0.0,-0.630509,0.0
…,…,…,…,…,…,…
128249,180,1,22874.0,0.0,1.962286,0.0
6010,1673,1,1078.0,0.0,0.871895,0.0
171321,2673,1,405.0,0.0,-1.791735,0.0


In [83]:
ctb_train

user_id,item_id,target,pop_rank,hist_rank,mf_rank,knn_rank
i64,i64,i32,f32,f32,f32,f32
290351,180,1,25116.0,0.0,2.028039,0.0
160238,298,1,15250.0,0.0,1.406034,0.0
945715,402,1,9897.0,0.0,0.112118,0.0
587820,3523,1,194.0,0.0,-2.173676,0.0
658149,156,1,30437.0,0.0,0.177681,0.0
…,…,…,…,…,…,…
1036696,362,1,11523.0,0.0,1.889153,0.0
321851,165,1,27716.0,0.0,2.072768,0.0
439129,168,1,27017.0,0.0,1.138532,0.0


class balance - weirdly disbalanced, will have to be fixed later

In [82]:
ctb_test['target'].value_counts()

target,count
i32,u32
1,3041
0,21


In [84]:
drop_col = ['user_id', 'item_id']
target_col = ['target']
cat_col = []

In [ ]:
print(type(ctb_train))

<class 'polars.dataframe.frame.DataFrame'>


In [85]:
X_train, y_train = ctb_train.drop(drop_col + target_col), ctb_train[target_col] #columns=
X_val, y_val = ctb_eval.drop(drop_col + target_col), ctb_eval[target_col] # columns=

In [86]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((550477, 4), (550477, 1), (58859, 4), (58859, 1))

In [88]:
X_train.head(), y_train.head()

(shape: (5, 4)
 ┌──────────┬───────────┬───────────┬──────────┐
 │ pop_rank ┆ hist_rank ┆ mf_rank   ┆ knn_rank │
 │ ---      ┆ ---       ┆ ---       ┆ ---      │
 │ f32      ┆ f32       ┆ f32       ┆ f32      │
 ╞══════════╪═══════════╪═══════════╪══════════╡
 │ 25116.0  ┆ 0.0       ┆ 2.028039  ┆ 0.0      │
 │ 15250.0  ┆ 0.0       ┆ 1.406034  ┆ 0.0      │
 │ 9897.0   ┆ 0.0       ┆ 0.112118  ┆ 0.0      │
 │ 194.0    ┆ 0.0       ┆ -2.173676 ┆ 0.0      │
 │ 30437.0  ┆ 0.0       ┆ 0.177681  ┆ 0.0      │
 └──────────┴───────────┴───────────┴──────────┘,
 shape: (5, 1)
 ┌────────┐
 │ target │
 │ ---    │
 │ i32    │
 ╞════════╡
 │ 1      │
 │ 1      │
 │ 1      │
 │ 1      │
 │ 1      │
 └────────┘)

#### Train Catboost

In [89]:
from catboost import CatBoostClassifier

# параметры для обучения
est_params = {
  'subsample': 0.9,
  'max_depth': 5,
  'n_estimators': 2000,
  'learning_rate': 0.1,
  'thread_count': 20,
  'random_state': 42,
  'verbose': 200,
}

ctb_model = CatBoostClassifier(**est_params)

In [90]:
X_train, y_train, X_val, y_val = X_train.to_numpy(), y_train.to_numpy(), X_val.to_numpy(), y_val.to_numpy() 


Catboost успешно обучен на кандидатах из датасета второй стадии. Обучение происходит на рангах 4-х моделей.

In [91]:
ctb_model.fit(X_train,
              y_train,
              eval_set=(X_val, y_val),
              early_stopping_rounds=300,
              cat_features=cat_col,
              plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

TBB Warning: The number of workers is currently limited to 10. The request for 19 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.




0:	learn: 0.4627991	test: 0.4639107	best: 0.4639107 (0)	total: 81.8ms	remaining: 2m 43s

200:	learn: 0.0455412	test: 0.0468035	best: 0.0468030 (163)	total: 3.26s	remaining: 29.2s

400:	learn: 0.0455412	test: 0.0468035	best: 0.0468030 (163)	total: 6.12s	remaining: 24.4s

Stopped by overfitting detector  (300 iterations wait)



bestTest = 0.04680300405

bestIteration = 163



Shrink model to first 164 iterations.


In [104]:
# save model  
with open(f"ctb_model.dill", 'wb') as f:
    dill.dump(ctb_model, f)

## <a id="test-solution"> </a>$\text{ Тесты работы baseline }$  

## Metrics

In [ ]:
from lightfm.evaluation import precision_at_k as lightfm_precision_at_k

In [ ]:
def precision_k(actual_items_long: pl.DataFrame, predicted_items: np.array, k=12):
    users = actual_items_long['user_id'].unique().to_list()

    user_interactions = actual_items_long.group_by('user_id').\
             agg(pl.col('item_id'))["item_id"].to_list()
    user_interactions = [set(i) for i in user_interactions]

    correct_predictions = np.array([
        len(user_interactions[i] & set(pop_items_k[j, :])) for (i, j) in zip(
            range(len(users)), range(len(predicted_items)) )
    ])

    precision_scores = correct_predictions / np.minimum(k, 
                            np.array([len(user_interactions[i]) for i in range(len(users))]))
    return np.mean(precision_scores)

In [ ]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if len( actual ) == 0:
        return 0.0

    return score / min(len(actual), k)


In [ ]:
def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## Test Base Models

In [ ]:
debug_set_list = all_user_purchases_from_sparse(debug_set_csr, item_ids)

In [ ]:
print("Popular:")

ks = [12,6,1]
pop_items_k = None

for k in ks:
    if pop_items_k is None:
        pop_items_k = pop_items[:k]
        pop_items_k = np.tile(pop_items_k, (len(debug_set_list), 1))
    else:
        pop_items_k = pop_items_k[:, :k]
    
    # prec = precision_k_pop(debug_set, pop_items_k, k=k)
    # print(f"Precision@{k} for POP: {prec}")
    mapkk = mapk(debug_set_list, pop_items_k, k=k)
    print(f"MAP@{k} for POP: {mapkk}")

Popular:

MAP@12 for POP: 2.1193276478428885e-05

MAP@6 for POP: 5.602758798432348e-07

MAP@1 for POP: 0.0


In [ ]:
def run_metrics(actual_long, actual_list, predicted, model_name="Model", ks=None):
    print(model_name, ':')
    if ks is None:
        ks = [1,6,12]
    
    for k in ks:
        prec = precision_k(actual_long, predicted, k=k)
        print(f"Precision@{k} for {model_name}: {prec}")
        mapkk = mapk(actual_list, predicted, k=k)
        print(f"MAP@{k} for {model_name}: {mapkk}")

        
run_metrics(debug_set, debug_set_list, hist_predictions_items, model_name="Historical")

## <a id="result-solution"> </a>$\text{ Выбор итогового решения для дальнейшей оптимизации и обоснования выбора }$ 

<div style="border-radius: 10px; border: #0000ff solid; padding: 15px; background-color: #f5f5f5; font-size: 16px; line-height: 1.6; text-align: left; box-sizing: border-box; overflow-wrap: break-word; height: auto; max-width: 1000px; margin: 20px auto; margin-top: 0; margin-bottom: 10px; color: #556B2F;">


Выбрана двух-этапная модель, сочетающая в себе на первом этапе:
* Popular Baseline 
* Historical Baseline 
* User-to-User KNN
* LightFM 
* Sequential - GRU4Rec

На втором этапе: 
CatBoostClassifier